# **Preparing SQLite Database**
After doing a bunch of data preparation in the previous notebooks, I'm ready to create a SQLite database with all of the data. 

# Setup
The cells below will help to set up the rest of the notebook. 

I'll start by configuring the kernel that's running this notebook:

In [1]:
# Change the cwd
%cd ..

# Enable the autoreload module
%load_ext autoreload
%autoreload 2

# Load the environment variables
from dotenv import load_dotenv
load_dotenv(override=True)

/Users/thubbard/Documents/personal/programming/pax-pal-2025/experiments


True

Next, I'm going to import the necessary modules:

In [2]:
# General imports
import os
import json

# Third-party imports
import pandas as pd

# Project-specific imports 
import utils.openai as openai_utils

# Loading Data
First off: I'll load in the data.

In [3]:
# Load in the playable_games_df
playable_games_df = pd.read_json("data/final_enriched_games_data.json")

# If any rows have an empty `description_texts` / `genres_and_tags`, drop them
playable_games_df = playable_games_df[
    playable_games_df["description_texts"].apply(
        lambda x: isinstance(x, list) and len(x) > 0
    )
    & playable_games_df["genres_and_tags"].apply(
        lambda x: isinstance(x, list) and len(x) > 0
    )
].copy()

/var/folders/p0/6dyp2b_d72z_d772nvm0y6tc0000gn/T/ipykernel_55797/1373856853.py:2: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  playable_games_df = pd.read_json("data/final_enriched_games_data.json")
/var/folders/p0/6dyp2b_d72z_d772nvm0y6tc0000gn/T/ipykernel_55797/1373856853.py:2: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  playable_games_df = pd.read_json("data/final_enriched_games_data.json")
/var/folders/p0/6dyp2b_d72z_d772nvm0y6tc0000gn/T/ipykernel_55797/1373856853.py:2: Futu

How many games do I have?

In [4]:
len(playable_games_df)

148

# Embedding Text
Before I actually create the SQLite database, I'm going to embed all of the relevant text. Specifically, I'll embed: 

- The average of each of the texts within `description_texts`
- The `snappy_summary`
- A comma-separated string of everything in `genres_and_tags`

I'll start by grabbing all of the text:

In [5]:
# Create a text_to_embed_df
text_to_embed_df_records = []
for row in playable_games_df.itertuples():
    # Find the longest description text
    longest_description = max(
        [desc_dict.get("text", "") for desc_dict in row.description_texts],
        key=len,
        default="",
    )

    # Create a single combined text
    combined_text = f"{row.name} {row.snappy_summary} {', '.join(row.genres_and_tags)} {longest_description}"

    text_to_embed_df_records.append(
        {"game_id": row.id, "emb_type": "combined", "text": combined_text}
    )
text_to_embed_df = pd.DataFrame(text_to_embed_df_records).dropna(subset=["text"])

# Drop anything where the text is not a string
text_to_embed_df = text_to_embed_df[
    text_to_embed_df["text"].apply(lambda x: isinstance(x, str) and len(x) > 0)
].copy()

Next, I'll embed everything:

In [6]:
embs = openai_utils.generate_embeddings_for_texts(
    text_list=text_to_embed_df["text"].tolist(), show_progress=True
)

# Add the embeddings to the dataframe
embs_df = text_to_embed_df.copy()
embs_df["emb"] = embs.tolist()

Generating Embeddings: 100%|██████████| 5/5 [00:01<00:00,  2.75it/s]


# Calculating Embedding Similarity

In [7]:
# Check the data types in embs_df
print(embs_df.dtypes)

# Convert embeddings from string to numpy arrays if needed
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import heapq

# Set the number of top similar games to retrieve
n = 6

# Make sure embeddings are numpy arrays
if isinstance(embs_df["emb"].iloc[0], str):
    embs_df["emb"] = embs_df["emb"].apply(lambda x: np.array(json.loads(x)))

# Extract game IDs and embeddings
game_ids = embs_df["game_id"].tolist()
embeddings = np.array(embs_df["emb"].tolist())

# Calculate pairwise cosine similarity
similarity_matrix = cosine_similarity(embeddings)

# Create a DataFrame to store the similarity matrix with game_ids as index and columns
similarity_df = pd.DataFrame(similarity_matrix, index=game_ids, columns=game_ids)

# For each game, find the top n most similar games (excluding itself)
top_similar_games = {}
for i, game_id in enumerate(game_ids):
    # Get similarities for this game (excluding itself)
    similarities = [
        (game_ids[j], similarity_matrix[i, j])
        for j in range(len(game_ids))
        if game_ids[j] != game_id
    ]

    # Sort by similarity score (descending) and take top n
    top_n = heapq.nlargest(n, similarities, key=lambda x: x[1])

    # Store as {game_id: [(similar_game_id, similarity_score), ...]}
    top_similar_games[game_id] = top_n

# Preview the results
print(f"Found similar games for {len(top_similar_games)} games")
sample_game_id = list(top_similar_games.keys())[0]
print(f"Example - Top similar games for {sample_game_id}:")
for similar_game_id, score in top_similar_games[sample_game_id]:
    print(f"  {similar_game_id}: similarity = {score:.4f}")

# Create a DataFrame with the similar games information
similar_games_records = []
for game_id, similar_games in top_similar_games.items():
    similar_games_dict = {
        "game_id": game_id,
        "similar_game_ids": [sg[0] for sg in similar_games],
        "similarity_scores": [float(sg[1]) for sg in similar_games],
    }
    similar_games_records.append(similar_games_dict)

similar_games_df = pd.DataFrame(similar_games_records)
print(f"\nSimilar games DataFrame shape: {similar_games_df.shape}")

# Create a dictionary mapping each game_id to its top similar games
similar_games_dict = {
    row["game_id"]: row["similar_game_ids"] for _, row in similar_games_df.iterrows()
}

game_id     object
emb_type    object
text        object
emb         object
dtype: object
Found similar games for 146 games
Example - Top similar games for ea4c3274bf51:
  31c4a8554ad9: similarity = 0.5651
  31970e15ae31: similarity = 0.5373
  19635cb48621: similarity = 0.5347
  0084af272678: similarity = 0.5155
  4f865f58714a: similarity = 0.5143
  3f83464714d6: similarity = 0.5121

Similar games DataFrame shape: (146, 3)


# Generating SQLite Database
Next up: I'm going to save all of the data in a SQLite database.

In [8]:
import sqlite3
import json
import sqlite_vec  # vec0 helper
import time
import os


# ─────────────────────────────────────────────────────────────────────────────
# Helper ─ execute_many with retry/back-off to dodge the "database is locked"
# ─────────────────────────────────────────────────────────────────────────────
def _execute_many_with_retry(
    cursor,
    sql: str,
    data,
    *,
    max_attempts: int = 5,
    initial_wait: float = 0.3,
) -> None:
    """
    Run `cursor.executemany(sql, data)` with exponential back-off so that the
    notebook doesn't crash when another process (or stray connection from a
    previous cell) is holding a write-lock on the SQLite file.

    Any non-locking OperationalError is re-raised immediately.
    """
    for attempt in range(max_attempts):
        try:
            cursor.executemany(sql, data)
            return  # ✅ success
        except sqlite3.OperationalError as exc:
            # Only retry if it's really a lock issue
            msg = str(exc).lower()
            if "database is locked" not in msg:
                raise

            wait_time = initial_wait * (2**attempt)
            print(
                f"SQLite is locked; retrying in {wait_time:.2f}s ({attempt+1}/{max_attempts})"
            )
            time.sleep(wait_time)

    # all retries exhausted
    raise RuntimeError(
        f"Could not run query after {max_attempts} attempts; still locked."
    )


# ─────────────────────────────────────────────────────────────────────────────
# Connection set-up
# ─────────────────────────────────────────────────────────────────────────────
db_path = "data/database.sqlite"

# Delete the database if it already exists to avoid schema conflicts
if os.path.exists(db_path):
    os.remove(db_path)
    print(f"Removed existing database at {db_path}")

with sqlite3.connect(
    db_path, timeout=30.0, isolation_level=None, check_same_thread=False
) as conn:
    conn.enable_load_extension(True)
    sqlite_vec.load(conn)  # load vec0 extension
    cursor = conn.cursor()

    # Use default journal mode instead of WAL
    cursor.execute("PRAGMA synchronous=NORMAL;")

    # ─────────────────────────────────────────────────────────────────────────
    # Schema
    # ─────────────────────────────────────────────────────────────────────────
    cursor.executescript(
        """
        CREATE TABLE IF NOT EXISTS games (
            id                TEXT PRIMARY KEY,
            name              TEXT,
            snappy_summary    TEXT,
            description_texts TEXT,
            platforms         TEXT,
            developer         TEXT,
            exhibitor         TEXT,
            booth_number      REAL,
            header_image_url  TEXT,
            steam_link        TEXT,
            genres_and_tags   TEXT,
            media             TEXT,
            released          REAL,
            release_time      TEXT,
            links             TEXT,
            similar_games     TEXT
        );

        CREATE VIRTUAL TABLE IF NOT EXISTS game_embs
        USING vec0(
            game_id TEXT PRIMARY KEY,
            vector  FLOAT[1536]
        );
        
        -- Full-text search index on human-readable text
        CREATE VIRTUAL TABLE IF NOT EXISTS games_fts
        USING fts5(
            id UNINDEXED,
            text
        );
    """
    )

    # ─────────────────────────────────────────────────────────────────────────
    # 1. Upsert games
    # ─────────────────────────────────────────────────────────────────────────
    game_records = [
        (
            row.id,
            row.name,
            getattr(row, "snappy_summary", None),
            json.dumps(row.description_texts),
            json.dumps(row.platforms),
            row.developer,
            row.exhibitor,
            row.booth_number,
            row.header_image_url,
            row.steam_link,
            json.dumps(row.genres_and_tags),
            json.dumps(row.media),
            row.released,
            row.release_time,
            json.dumps(getattr(row, "links", None)),
            json.dumps(similar_games_dict.get(row.id, [])),
        )
        for row in playable_games_df.itertuples()
    ]
    _execute_many_with_retry(
        cursor,
        """
        INSERT OR REPLACE INTO games (
            id, name, snappy_summary, description_texts, platforms,
            developer, exhibitor, booth_number, header_image_url, steam_link,
            genres_and_tags, media, released, release_time, links, similar_games
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """,
        game_records,
    )

    # ─────────────────────────────────────────────────────────────────────────
    # 2. Upsert embeddings (avoid UNIQUE constraint conflicts)
    # ─────────────────────────────────────────────────────────────────────────
    # clear out any stale rows
    cursor.execute("DELETE FROM game_embs")

    # drop any accidental duplicates in the DataFrame
    deduped = embs_df.drop_duplicates(subset="game_id", keep="last")
    emb_records = [
        (row["game_id"], json.dumps(row["emb"])) for _, row in deduped.iterrows()
    ]

    _execute_many_with_retry(
        cursor,
        "INSERT OR REPLACE INTO game_embs (game_id, vector) VALUES (?, ?)",
        emb_records,
    )

    # ─────────────────────────────────────────────────────────────────────────
    # 3. Populate full-text search index
    # ─────────────────────────────────────────────────────────────────────────
    cursor.execute("DELETE FROM games_fts")

    # Prepare records for FTS table
    fts_records = []
    for _, row in embs_df.iterrows():
        fts_records.append((row["game_id"], row["text"]))

    _execute_many_with_retry(
        cursor,
        """
        INSERT INTO games_fts (id, text)
        VALUES (?, ?)
        """,
        fts_records,
    )

print(
    f"Database created successfully with "
    f"{len(playable_games_df)} games and {len(embs_df)} embeddings 🎮"
)

Removed existing database at data/database.sqlite
Database created successfully with 148 games and 148 embeddings 🎮


# Reading from SQLite Database

In [ ]:
import sqlite3
import sqlite_vec
import struct
import json  # For potentially pretty-printing game details if needed

# Database path
db_path = "data/database.sqlite"

# Dimension of your embeddings (must match what you stored)
# From your schema: CREATE VIRTUAL TABLE ... vector FLOAT[1536]
embedding_dim = 1536

# List to store the results
results = []

# Connect to the database
with sqlite3.connect(db_path) as conn:
    # Enable extension loading and load sqlite-vec
    # Although not strictly needed for reading blobs,
    # it's good practice if you might do vector operations later.
    conn.enable_load_extension(True)
    sqlite_vec.load(conn)
    conn.enable_load_extension(False)  # Disable after loading

    cursor = conn.cursor()

    # Query to get top 3 games by booth number (desc) and their embeddings
    query = """
    SELECT
        g.id,
        g.name,
        g.booth_number,
        ge.vector  -- This will be a BLOB
    FROM
        games g
    JOIN
        game_embs ge ON g.id = ge.game_id
    WHERE
        g.booth_number IS NOT NULL -- Ensure we only sort valid numbers
    ORDER BY
        g.booth_number DESC
    LIMIT 3;
    """

    cursor.execute(query)
    rows = cursor.fetchall()

    # Process the results
    for row in rows:
        game_id, name, booth_number, embedding_blob = row

        # Deserialize the embedding blob
        # The format string '<{embedding_dim}f' means:
        # '<' = little-endian
        # '{embedding_dim}' = number of floats (e.g., 1536)
        # 'f' = float (4 bytes)
        try:
            # Use struct.unpack to convert bytes back to tuple of floats
            embedding_tuple = struct.unpack(f"<{embedding_dim}f", embedding_blob)
            # Convert tuple to list
            embedding_list = list(embedding_tuple)
        except struct.error as e:
            print(f"Error unpacking embedding for game {game_id}: {e}")
            print(
                f"Blob length: {len(embedding_blob)} bytes. Expected: {embedding_dim * 4} bytes."
            )
            embedding_list = None  # Indicate failure

        results.append(
            {
                "id": game_id,
                "name": name,
                "booth_number": booth_number,
                "embedding": embedding_list,  # Now it's a list of floats
            }
        )

# Print the results
for game in results:
    # Print embedding length or first few elements for brevity
    emb_preview = game["embedding"][:5] if game["embedding"] else "Error/None"
    print(f"Game ID: {game['id']}")
    print(f"Name: {game['name']}")
    print(f"Booth #: {game['booth_number']}")
    print(f"Embedding (first 5): {emb_preview}...")
    print("-" * 10)